# Retrieval Augmented Generation (RAG) Server

## 1. Setup

**Instructions:**

a) Download model

```bash
huggingface-cli download hkunlp/instructor-large \
    --revision 54e5ffb8d484de506e59443b07dc819fb15c7233 \
    --local-dir ~/.gai/models/instructor-large \
    --local-dir-use-symlinks False
```


---

## 2. Load test configuration

In [1]:
from gai.lib.server.singleton_host import SingletonHost
from gai.lib.common.utils import free_mem
from rich.console import Console
console=Console()

config = {
    "type": "rag",
    "generator_name": "instructor-sentencepiece",
    "chromadb": {
        "path": "rag/chromadb",
        "n_results": 3
    },
    "sqlite": {
        "path": "rag/gai-rag.db"
    },
    "model_path": "models/instructor-large",
    "device": "cuda",
    "chunks": {
        "size": 1000,
        "overlap": 100,
        "path": "chunks"
    },
    "module_name": "gai.rag.server.gai_rag",
    "class_name": "RAG",
    "init_args": [],
    "init_kwargs": {}
}


## 3. Load Model Test

In [2]:
# before loading
free_mem()
try:
    with SingletonHost.GetInstanceFromConfig(config) as host:

        # after loading
        free_mem()
except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()
    

Free memory: 5.56 GB

INFO     RAG: device=cuda
INFO     RAG: sqlite=sqlite:///:memory:


load INSTRUCTOR_Transformer


/home/kakkoii1337/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


max_seq_length  512


                         Performance load                          
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 1.39 seconds │ 1.39 seconds │
│ CPU Memory        │          - MB │      5.66 MB │      5.66 MB │
│ CUDA Memory       │       5.59 GB │      0.01 GB │      5.58 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

Free memory: 5.58 GB

Free memory: 5.57 GB

## Indexing

In [3]:
from gai.rag.server.gai_rag import RAG
from gai.rag.dtos.create_doc_header_request import CreateDocHeaderRequestPydantic
try:
    with SingletonHost.GetInstanceFromConfig(config) as host:
        rag = host.generator

        req = CreateDocHeaderRequestPydantic(
            CollectionName='demo',
            FileType='txt',
            Source='https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            Title='2023 National Day Rally Speech',
            FilePath="./pm_long_speech_2023.txt"
        )


        # Index
        doc_id = await rag.index_async(
            req=req
            # collection_name='demo',
            # file_type='txt',
            # source="https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech",
            # title="2023 National Day Rally Speech"
            )

except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()


INFO     RAG: device=cuda


INFO     RAG: sqlite=sqlite:///:memory:


load INSTRUCTOR_Transformer
max_seq_length  512


                         Performance load                          
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 0.80 seconds │ 0.80 seconds │
│ CPU Memory        │          - MB │      1.49 MB │      1.49 MB │
│ CUDA Memory       │       5.56 GB │     -0.00 GB │      5.57 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

INFO     rag.index_document_header_async: request started. collection_name=demo file_path=./pm_long_speech_2023.txt title=2023 National Day Rally Speech source=https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech abstract=None authors=None publisher=None published_date=None comments=None keywords=None
DEBUG    rag.index_document_header_async: creating doc header with id=PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U.
DEBUG    rag.index_document_header_async: document_header created. id=PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U
INFO     rag.index_document_split_async: splitting chunks
INFO     rag.index_document_split_async: chunkgroup created. chunkgroup_id=e5d1e3bc-bd67-42e4-93f9-126835d5f0f9
100%|██████████| 66/66 [00:00<00:00, 487.85it/s]
INFO     rag.index_document_split_async: chunks created. count=66
INFO     RAG.index_document_index_async: Start indexing...
0it [00:00, ?it/s]DEBUG    RAG.index_document_index_async: Indexed 1/66 chunk 2e3b8ed1-bf06-4ad9-8b03-3a6f16f537f0

Free memory: 5.44 GB

In [4]:
try:
    with SingletonHost.GetInstanceFromConfig(config) as host:
        rag=host.generator
        # Index
        result = rag.retrieve(collection_name="demo",query_texts="Who are the young seniors?")
        console.print(result)

except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()


INFO     RAG: device=cuda
INFO     RAG: sqlite=sqlite:///:memory:


load INSTRUCTOR_Transformer
max_seq_length  512


                         Performance load                          
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 0.87 seconds │ 0.87 seconds │
│ CPU Memory        │          - MB │      1.50 MB │      1.50 MB │
│ CUDA Memory       │       5.43 GB │     -0.00 GB │      5.43 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

INFO     RAG.retrieve: Retrieving by query Who are the young seniors?...


[
    {
        'documents': 'Especially for those in their 50s and early 60s. Let us call them the “Young Seniors”. 
"Young”, because you are younger than the Pioneer Generation and the Merdeka Generation; “Seniors”, because you 
will soon retire, or maybe you have recently retired.',
        'metadatas': {
            'Abstract': '',
            'ChunkGroupId': 'e5d1e3bc-bd67-42e4-93f9-126835d5f0f9',
            'DocumentId': 'PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U',
            'Keywords': '',
            'PublishedDate': '',
            'Source': 'https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            'Title': '2023 National Day Rally Speech'
        },
        'distances': 0.09020859003067017,
        'ids': 'e1ba2adb-bbc9-4190-bae2-085a2129c081'
    },
    {
        'documents': 'Young Seniors are in a unique position today. Compared to the Pioneer and Merdeka 
Generations, you have benefited more from Singapore’s growth, and generally done better in life. But compared to 
workers younger than you, in their 30s and 40s today, you have generally earned less over your lifetimes. You have 
also had less time to benefit from improvements to the CPF system, and so have built up less retirement savings. 
Young Seniors are also in a particularly sandwiched phase of your lives. You have to shoulder the responsibility of
caring for both the young and old in your families. Your kids may be young adults, but often are not yet fully 
independent and still live in the same household. Many of you Young Seniors also have elderly parents at home, who 
may be beset with the infirmities of old age. You have to shuttle them to medical appointments and hospital visits,
plus attend to their everyday needs. All this, while watching your own health, because you are not so youthful',
        'metadatas': {
            'Abstract': '',
            'ChunkGroupId': 'e5d1e3bc-bd67-42e4-93f9-126835d5f0f9',
            'DocumentId': 'PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U',
            'Keywords': '',
            'PublishedDate': '',
            'Source': 'https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            'Title': '2023 National Day Rally Speech'
        },
        'distances': 0.11518943309783936,
        'ids': '1292ade9-df83-460d-8e3b-fef85d9eac3e'
    },
    {
        'documents': 'to their everyday needs. All this, while watching your own health, because you are not so 
youthful yourselves. Responsibilities on all these fronts multiply your burdens. So, as Young Seniors, you are 
understandably anxious about your retirement needs. Beyond the daily cost of living pressures, you know that 
retirement is creeping up on you. You wonder: Will I have enough to get by? Can I cope? But do not worry – the 
Government will help you. You will not be left behind.',
        'metadatas': {
            'Abstract': '',
            'ChunkGroupId': 'e5d1e3bc-bd67-42e4-93f9-126835d5f0f9',
            'DocumentId': 'PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U',
            'Keywords': '',
            'PublishedDate': '',
            'Source': 'https://www.pmo.gov.sg/Newsroom/2023-National-Day-Rally-Speech',
            'Title': '2023 National Day Rally Speech'
        },
        'distances': 0.11691617965698242,
        'ids': 'b6914df9-dd0b-4937-83fc-75c817887d14'
    }
]

Free memory: 5.44 GB

## API

#### a) List Collections

In [5]:
%%bash
curl -s http://localhost:12036/gen/v1/rag/collections

{"collections":[]}

#### b) Delete collection

In [6]:
%%bash
curl -s -X DELETE http://localhost:12036/gen/v1/rag/collection/demo

{"detail":{"code":"collections_not_found","message":"Collection demo not found"}}

#### c) index

In [7]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/index-file' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'file=@./pm_long_speech_2023.txt' \
    -F 'req={"CollectionName":"demo","Source": "https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023","FilePath":"./pm_long_speech_2023.txt"}'

{"document_id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","chunkgroup_id":"ea55e9d0-5c25-4dcf-a1dd-1e4f174b25f0","chunk_ids":["0bed4d2d-567e-4abe-a6a0-ac49677713e0","3da8329c-58d4-416c-a1d2-7148783b6710","6dd48833-d859-42e2-8ca8-793d03eeb127","08c901bc-4d20-403e-bfb3-6f3e0f8768cb","bfe0a7e9-6077-4d48-b2ab-8a7f05634b99","6859d0bc-1db9-475e-b36d-8165b9b7214b","da7495d3-1645-40a2-8935-ebb32f7c26a8","0d06e3bb-c43e-4d5e-8c57-af0373c2a591","d5ce0371-4772-4b8b-a010-b00395e21908","f567895e-26ed-4db5-8df1-fa763af739f1","4959ae2e-43f7-408e-8469-fbf48a31d64a","62e3ff8b-f9f0-4cad-9f3e-2c00e36b82cb","2803f51e-d422-42cb-8651-9d82a204181c","57d0f3cd-a3e1-4894-ab84-976fca435162","83227d57-c9a1-4df4-88c4-c8428846c728","4b19da5e-29da-45b7-8e54-7cd625ec7145","9c60299a-147e-448b-86e4-786d2b847fe9","a40bd1cb-1806-4a46-ad0c-dd243dba01ac","18ae8c41-67f0-4074-8e31-37246b3bfe84","341a35da-f32e-497e-a1ed-ee6f81f13eda","34e5b929-a501-42d4-93b4-e7c25c1f18c4","45fdfbad-baaf-4de5-ae6e-0346abee8bdb","7ad5535b-8b1

#### c) index step-by-step (method 2)

In [2]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/step/header' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'file=@./pm_long_speech_2023.txt' \
    -F 'req={"CollectionName":"demo","Source": "https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023","FilePath":"./pm_long_speech_2023.txt"}'

{"Id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","CollectionName":"demo","ByteSize":43352,"FileName":"pm_long_speech_2023.txt","FileType":".txt","Source":"https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023","Abstract":null,"Authors":null,"Title":null,"Publisher":null,"PublishedDate":null,"Comments":null,"Keywords":null,"CreatedAt":"2024-09-05T00:22:19.604837","UpdatedAt":"2024-09-05T00:23:03.116357","IsActive":true,"ChunkGroups":[]}

#### d) verify document exists

In [8]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/collection/demo/document/exists' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'file=@./pm_long_speech_2023.txt' 

{"document":{"Id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","CollectionName":"demo","ByteSize":43352,"FileName":"pm_long_speech_2023.txt","FileType":".txt","File":"PM Lee Hsien Loong delivered his National Day Rally speech on 20 August 2023 at the Institute of Technical Education (ITE) Headquarters. PM spoke in Malay and Chinese, followed by English.\n\n \n“A BETTER HOME, A BRIGHTER FUTURE”\nSECTION 1: INTRODUCTION\nMy fellow Singaporeans, good evening.\n\nReturn to Normalcy\nWe are all relieved that COVID is behind us. Life as we knew it has resumed.\n\nCOVID-19 was the most challenging ordeal for our nation since Independence. We can all be proud of how we pulled through together. Unlike many other countries, Singapore has emerged from the pandemic stronger, more resilient, and more united. This is a tribute to the indomitable spirit of our nation.\n\nGlobal situation and economic outlook\nHaving come through COVID, we are once again being tested. The international environment is

#### e) retrieve

In [9]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/retrieve' \
    -s \
    -H "Content-Type: application/json" \
    -d '{"collection_name":"demo","query_texts":"Who are the young seniors?","n_results":4}'


{"retrieved":[{"documents":"Especially for those in their 50s and early 60s. Let us call them the “Young Seniors”. \"Young”, because you are younger than the Pioneer Generation and the Merdeka Generation; “Seniors”, because you will soon retire, or maybe you have recently retired.","metadatas":{"Abstract":"","ChunkGroupId":"ea55e9d0-5c25-4dcf-a1dd-1e4f174b25f0","DocumentId":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","Keywords":"","PublishedDate":"","Source":"https://www.pmo.gov.sg/Newsroom/National-Day-Rally-2023","Title":""},"distances":0.09020859003067017,"ids":"f17ff36a-ab58-4b15-b5b6-9349aa3ef87c"},{"documents":"Young Seniors are in a unique position today. Compared to the Pioneer and Merdeka Generations, you have benefited more from Singapore’s growth, and generally done better in life. But compared to workers younger than you, in their 30s and 40s today, you have generally earned less over your lifetimes. You have also had less time to benefit from improvements to the CPF syst

g) list documents

In [10]:
%%bash
curl -s 'http://localhost:12036/gen/v1/rag/collection/demo/documents'

{"documents":[{"Id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","CollectionName":"demo","ByteSize":43352,"FileName":"pm_long_speech_2023.txt","FileType":".txt","File":"PM Lee Hsien Loong delivered his National Day Rally speech on 20 August 2023 at the Institute of Technical Education (ITE) Headquarters. PM spoke in Malay and Chinese, followed by English.\n\n \n“A BETTER HOME, A BRIGHTER FUTURE”\nSECTION 1: INTRODUCTION\nMy fellow Singaporeans, good evening.\n\nReturn to Normalcy\nWe are all relieved that COVID is behind us. Life as we knew it has resumed.\n\nCOVID-19 was the most challenging ordeal for our nation since Independence. We can all be proud of how we pulled through together. Unlike many other countries, Singapore has emerged from the pandemic stronger, more resilient, and more united. This is a tribute to the indomitable spirit of our nation.\n\nGlobal situation and economic outlook\nHaving come through COVID, we are once again being tested. The international environment 

h) get document

In [11]:
%%bash
curl -s 'http://localhost:12036/gen/v1/rag/collection/demo/document/PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U'

{"document":{"Id":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U","CollectionName":"demo","ByteSize":43352,"FileName":"pm_long_speech_2023.txt","FileType":".txt","File":"PM Lee Hsien Loong delivered his National Day Rally speech on 20 August 2023 at the Institute of Technical Education (ITE) Headquarters. PM spoke in Malay and Chinese, followed by English.\n\n \n“A BETTER HOME, A BRIGHTER FUTURE”\nSECTION 1: INTRODUCTION\nMy fellow Singaporeans, good evening.\n\nReturn to Normalcy\nWe are all relieved that COVID is behind us. Life as we knew it has resumed.\n\nCOVID-19 was the most challenging ordeal for our nation since Independence. We can all be proud of how we pulled through together. Unlike many other countries, Singapore has emerged from the pandemic stronger, more resilient, and more united. This is a tribute to the indomitable spirit of our nation.\n\nGlobal situation and economic outlook\nHaving come through COVID, we are once again being tested. The international environment is

i) update document

In [12]:
%%bash
curl -X PUT \
    http://localhost:12036/gen/v1/rag/collection/demo/document/PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U \
    -H 'Content-Type: application/json' \
    -s \
    -d '{
            "Publisher": "ABC"
        }'


{"message":"Document updated successfully","document":"PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U"}

j) delete document

In [13]:
%%bash
curl -s -X DELETE http://localhost:12036/gen/v1/rag/collection/demo/document/PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U

{"message":"Document with id PwR6VmXqAfwjn84ZM6dePsLWTldPv8cNS5dESYlsY2U deleted successfully"}

---
#### Example : Index PDF using Multi-Step Indexing

The purpose for multi-step is to support interactive status update so that the client can make use of the websocket manager to get the status of the indexing process in a step-by-step manner.

NOTE: The same is achievable using the single-step indexing as well as before.

```
curl -X POST 'http://localhost:12036/gen/v1/rag/index-file' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'collection_name=demo' \
    -F 'file=@./attention-is-all-you-need.pdf' \
    -F 'req={"CollectionName":"demo","Source": "https://arxiv.org/abs/1706.03762","FilePath":"attention-is-all-you-need.pdf"}'
```


##### Step 1: Index document header

In [35]:
%%bash
curl -X POST 'http://localhost:12036/gen/v1/rag/step/header' \
    -H 'accept: application/json' \
    -H 'Content-Type: multipart/form-data' \
    -s \
    -F 'file=@./attention-is-all-you-need.pdf' \
    -F 'req={"CollectionName":"demo","Source": "https://arxiv.org/abs/1706.03762","FilePath":"attention-is-all-you-need.pdf"}'

{"Id":"-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0","CollectionName":"demo","ByteSize":2215244,"FileName":"attention-is-all-you-need.pdf","FileType":".pdf","File":null,"Source":"https://arxiv.org/abs/1706.03762","Abstract":null,"Authors":null,"Title":null,"Publisher":null,"PublishedDate":null,"Comments":null,"Keywords":null,"CreatedAt":"2024-09-05T01:44:49.849553","UpdatedAt":"2024-09-05T01:44:49.849564","IsActive":true,"ChunkGroups":[]}

##### Step 2: Split document into a group of chunks

In [36]:
import json
import os

# Execute the curl command and capture the response
response = !curl -X POST 'http://localhost:12036/gen/v1/rag/step/split' \
    -H 'Content-Type: application/json' \
    -s \
    -d '{\
            "collection_name": "demo",\
            "document_id": "-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0",\
            "chunk_size": 1000,\
            "chunk_overlap": 100\
        }'

# Convert response to string, then load it as JSON
response_json = json.loads(''.join(response))

# Extract the Id
document_id = response_json["Id"]
print(document_id)

# Set environment variable
os.environ['DOCUMENT_ID'] = document_id


17ab24db-1ea4-4fc1-bae2-698df2068e13


##### Step 3: Index each chunk in the database

In [37]:
%%bash
echo $DOCUMENT_ID
curl -X POST 'http://localhost:12036/gen/v1/rag/step/index' \
    -H 'Content-Type: application/json' \
    -s \
    -d '{
            "collection_name": "demo",
            "document_id": "-Sc9eXzUiSlaFV3qEDaKam33Boamkvv4tea8YPsjpy0",
            "chunkgroup_id": "'$DOCUMENT_ID'"
        }'

17ab24db-1ea4-4fc1-bae2-698df2068e13


{"chunk_ids":["bd195379-81ae-48a3-8666-a193756e121c","8ff2ec50-3bb7-4639-9da3-cbb4cae3482e","561e1ea2-a9d8-4857-9800-0abecb0d84bf","92fa074f-a4c4-488c-9fb7-b07365f5a53b","1cf78885-c926-4390-ae2d-435696f74d2d","bb222005-174e-47de-a5c7-329bef6380e1","6715da3c-0238-43a4-90b5-612fd9467d4e","bba19556-11e7-40e6-b444-043b9d85c589","f77468fa-a302-422d-9d7c-7284855cb0d4","99124602-20be-4eab-a296-a9e537a257f8","d2c2e01c-95ed-41bc-9a0f-f5f3ec8eeb16","3e610487-1b14-4dee-92dd-0213d0241eca","507558d5-257b-4437-b39b-e557c4e58f5b","94dc9e93-3148-402e-a4d8-0b6cbf581571","2ac4ac47-e8b9-4020-b87f-8b08cb37d716","e452e91c-c4d4-4b77-bde3-2300dd093e9c","d8371791-1636-403f-8218-0b1fb9ba1783","43d5c392-bc5e-48be-b696-52d863bcef0f","1d8a47cd-134d-4f06-a657-fe856cc7a0f1","983e25f5-9108-4975-8b05-d018ae4336f9","20f1e559-506d-43ea-a56a-e19f4e656c59","1318da34-8186-45a4-ae58-1715483535ae","26ee7b2e-e564-49a3-b8e1-60f0bf96e35c","43e72e92-9e08-4ad9-8b75-f8d46ddcfaff","ded714f6-d2d3-44d9-9342-1c973fe0eaea","06c66333-e

---

## 5. Docker

**Instructions:** 

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker-compose: up**

#### Smoke Test

In [ ]:
%%bash
curl http://localhost:12036


**Tests:**

Repeat the API test (#)

**Tear Down:**

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker-compose: down**

### Debugging

a) Container must be started with "python -m debugpy --listen 0.0.0.0:5678 main.py"

b) Port 5678 must be opened.

c) Click on "Debug" in Tool bar

d) Select "Attach" > "Run and Debug"

e) Add a "breakpoint" in the code

f) Run the API test to see if it trigger the breakpoint.